In [1]:
import os
import numpy as np
import pandas as pd
import multiprocessing
from sklearn import svm
from pathlib import Path
import matplotlib.pyplot as plt
from keras.preprocessing import image
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.image import ImageDataGenerator

In [2]:
cpus = multiprocessing.cpu_count()
print(cpus)

4


In [3]:
mapping = pd.read_csv('train.csv').values

In [4]:
mapping

array([['296.jpg', 'Pikachu'],
       ['297.jpg', 'Pikachu'],
       ['298.jpg', 'Pikachu'],
       ['299.jpg', 'Pikachu'],
       ['300.jpg', 'Pikachu'],
       ['301.jpg', 'Pikachu'],
       ['302.jpg', 'Pikachu'],
       ['303.jpg', 'Pikachu'],
       ['304.jpg', 'Pikachu'],
       ['305.jpg', 'Pikachu'],
       ['306.jpg', 'Pikachu'],
       ['307.jpg', 'Pikachu'],
       ['308.jpg', 'Pikachu'],
       ['309.jpg', 'Pikachu'],
       ['310.jpg', 'Pikachu'],
       ['311.jpg', 'Pikachu'],
       ['312.jpg', 'Pikachu'],
       ['313.jpg', 'Pikachu'],
       ['314.jpg', 'Pikachu'],
       ['315.jpg', 'Pikachu'],
       ['316.jpg', 'Pikachu'],
       ['317.jpg', 'Pikachu'],
       ['318.jpg', 'Pikachu'],
       ['319.jpg', 'Pikachu'],
       ['320.jpg', 'Pikachu'],
       ['321.jpg', 'Pikachu'],
       ['322.jpg', 'Pikachu'],
       ['323.jpg', 'Pikachu'],
       ['324.jpg', 'Pikachu'],
       ['325.jpg', 'Pikachu'],
       ['326.jpg', 'Pikachu'],
       ['327.jpg', 'Pikachu'],
       [

In [5]:
pairs = dict()
for i in mapping:
    pairs[i[0]] = i[1]

In [6]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

In [7]:
dirs = Path("Images/").glob("*jpg")

labels_dict = {"Bulbasaur":0,"Pikachu":1,"Charmander":2}

image_data = []
labels = []

for image_path in dirs:
    label = str(image_path).split("\\")[-1]
#     print(label)
    if label in pairs:
        img = image.load_img("Images/" + label,target_size=(32,32))
        img_array = image.img_to_array(img)
        norm = np.linalg.norm(img_array)
        img_array = img_array / norm
        image_data.append(img_array)
        labels.append(labels_dict[pairs[label]])

C:\Users\rrite\anaconda3\lib\site-packages\PIL\Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [8]:
image_data = np.array(image_data,dtype='float32')/255.0
labels = np.array(labels)

In [9]:
M = image_data.shape[0] 
image_data = image_data.reshape(M,-1)

In [10]:
params = [
    {
        'kernel':['linear','rbf','poly','sigmoid'],
        'C':[
             0.160, 0.161, 0.162, 0.163, 0.164, 0.165, 0.166, 0.167, 0.168, 0.169,
             0.170, 0.171, 0.172, 0.173, 0.174, 0.175, 0.176, 0.177, 0.178, 0.179,
             0.180, 0.181, 0.182, 0.183, 0.184, 0.185, 0.186, 0.187, 0.188, 0.189,
             0.190, 0.191, 0.192, 0.193, 0.194, 0.195, 0.196, 0.197, 0.198, 0.199,
             ]
    }   
]

In [17]:
svm_classifier = svm.SVC(kernel='poly',C=0.167)

In [18]:
svm_classifier.fit(image_data,labels)
svm_classifier.score(image_data,labels)

1.0

In [13]:
gs = GridSearchCV(estimator=svm.SVC(),param_grid=params,scoring="accuracy",cv=5,n_jobs = cpus)

In [14]:
gs.fit(image_data, labels)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=4,
             param_grid=[{'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08,
                                0.09, 0.1, 0.11, 0.111, 0.112, 0.113, 0.114,
                                0.115, 0.116, 0.117, 0.118, 0.119, 0.12, 0.121,
                                0.122, 0.123, 0.124, 0.125, 0.126, 0.127, 0.128,
                                0.129, ...],
                          'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}],
             scoring='accuracy')

In [15]:
gs.best_estimator_

SVC(C=0.167, kernel='poly')

In [16]:
gs.best_score_

0.8846448087431694

### Testing on test data and generating csv output for submission

In [19]:
test_mapping = pd.read_csv('test.csv')

In [20]:
test_mapping.head()

,ImageId
0,1106.jpg
1,2236.jpg
2,1131.jpg
3,2221.jpg
4,2011.jpg


In [22]:
test_mapping = test_mapping.values

In [23]:
test_mapping[0]

array(['1106.jpg'], dtype=object)

In [41]:
test_labels_dict = {0: "Bulbasaur", 1: "Pikachu",2: "Charmander"}

test_image_data = []
labels = []

for item in test_mapping:
    img = image.load_img("Images/" + item[0], target_size=(32,32))
    img_array = image.img_to_array(img)
    norm = np.linalg.norm(img_array)
    img_array = img_array / norm
    test_image_data.append(img_array)


In [42]:
test_image_data = np.array(test_image_data, dtype='float32') / 255.0

In [49]:
print(test_image_data.shape, test_mapping.size)

(123, 3072) 123


In [45]:
M_ = test_image_data.shape[0] 
test_image_data = test_image_data.reshape(M_,-1)

In [47]:
test_image_data

array([[9.2303715e-05, 9.2303715e-05, 9.2303715e-05, ..., 9.2303715e-05,
        9.2303715e-05, 9.2303715e-05],
       [8.0646918e-05, 8.0646918e-05, 8.0646918e-05, ..., 8.0646918e-05,
        8.0646918e-05, 8.0646918e-05],
       [9.8270735e-05, 9.8270735e-05, 9.8270735e-05, ..., 9.8270735e-05,
        9.8270735e-05, 9.8270735e-05],
       ...,
       [7.5248776e-05, 7.5248776e-05, 7.5248776e-05, ..., 7.5248776e-05,
        7.5248776e-05, 7.5248776e-05],
       [2.0718835e-05, 5.5250228e-05, 4.6185738e-05, ..., 7.4242496e-05,
        8.9349975e-05, 3.8416172e-05],
       [6.2177380e-05, 6.2177380e-05, 6.2177380e-05, ..., 8.5565207e-06,
        8.5565207e-06, 8.5565207e-06]], dtype=float32)

In [60]:
output = []

In [55]:
test_labels_dict[svm_classifier.predict(test_image_data[0].reshape(1, -1))[0]]

'Bulbasaur'

In [61]:
for i in range(test_mapping.size):
    output.append([test_mapping[i][0], test_labels_dict[svm_classifier.predict(test_image_data[i].reshape(1, -1))[0]]])

In [62]:
output

[['1106.jpg', 'Bulbasaur'],
 ['2236.jpg', 'Charmander'],
 ['1131.jpg', 'Bulbasaur'],
 ['2221.jpg', 'Charmander'],
 ['2011.jpg', 'Pikachu'],
 ['2030.jpg', 'Pikachu'],
 ['2035.jpg', 'Pikachu'],
 ['1110.jpg', 'Bulbasaur'],
 ['1104.jpg', 'Bulbasaur'],
 ['2217.jpg', 'Charmander'],
 ['1111.jpg', 'Bulbasaur'],
 ['1100.jpg', 'Bulbasaur'],
 ['1112.jpg', 'Bulbasaur'],
 ['1138.jpg', 'Bulbasaur'],
 ['2238.jpg', 'Charmander'],
 ['2009.jpg', 'Bulbasaur'],
 ['2225.jpg', 'Charmander'],
 ['2240.jpg', 'Charmander'],
 ['2020.jpg', 'Pikachu'],
 ['2209.jpg', 'Charmander'],
 ['2029.jpg', 'Pikachu'],
 ['1114.jpg', 'Bulbasaur'],
 ['2201.jpg', 'Charmander'],
 ['1118.jpg', 'Bulbasaur'],
 ['2025.jpg', 'Charmander'],
 ['2202.jpg', 'Charmander'],
 ['1117.jpg', 'Bulbasaur'],
 ['1119.jpg', 'Bulbasaur'],
 ['2000.jpg', 'Pikachu'],
 ['2007.jpg', 'Pikachu'],
 ['2039.jpg', 'Pikachu'],
 ['2017.jpg', 'Pikachu'],
 ['2215.jpg', 'Charmander'],
 ['1120.jpg', 'Bulbasaur'],
 ['2006.jpg', 'Pikachu'],
 ['1116.jpg', 'Bulbasaur'],
 

In [65]:
output_1 = np.array(output)

In [66]:
output_1

array([['1106.jpg', 'Bulbasaur'],
       ['2236.jpg', 'Charmander'],
       ['1131.jpg', 'Bulbasaur'],
       ['2221.jpg', 'Charmander'],
       ['2011.jpg', 'Pikachu'],
       ['2030.jpg', 'Pikachu'],
       ['2035.jpg', 'Pikachu'],
       ['1110.jpg', 'Bulbasaur'],
       ['1104.jpg', 'Bulbasaur'],
       ['2217.jpg', 'Charmander'],
       ['1111.jpg', 'Bulbasaur'],
       ['1100.jpg', 'Bulbasaur'],
       ['1112.jpg', 'Bulbasaur'],
       ['1138.jpg', 'Bulbasaur'],
       ['2238.jpg', 'Charmander'],
       ['2009.jpg', 'Bulbasaur'],
       ['2225.jpg', 'Charmander'],
       ['2240.jpg', 'Charmander'],
       ['2020.jpg', 'Pikachu'],
       ['2209.jpg', 'Charmander'],
       ['2029.jpg', 'Pikachu'],
       ['1114.jpg', 'Bulbasaur'],
       ['2201.jpg', 'Charmander'],
       ['1118.jpg', 'Bulbasaur'],
       ['2025.jpg', 'Charmander'],
       ['2202.jpg', 'Charmander'],
       ['1117.jpg', 'Bulbasaur'],
       ['1119.jpg', 'Bulbasaur'],
       ['2000.jpg', 'Pikachu'],
       ['2007.jp

In [67]:
out_df = pd.DataFrame(output_1, columns=['ImageId', 'NameOfPokemon'])
out_df.to_csv('outputfile.csv', index=False, index_label=False)